In [1]:
import numpy as np
import copy
import itertools
from scipy import linalg as sLA
from numpy import linalg as LA
import time
from numba import njit
from numba import jit


import sys
sys.path.append("../scripts")

from Gates import *

In [2]:
class Qpsi():
    def __init__(self, N = 2):
        self.N = N
        self.coefs = None
    def build_random_vec(self):
        self.coefs = np.random.rand(2**self.N) + 1j*np.random.rand(2**self.N)
        
    def set_coefs(self, coefs):
        if len(coefs) == 2**self.N:
            self.coefs = np.arryay(coefs)
            
    def get_coefs(self):
        return self.coefs
    
    def apply_U(self, U, axis):
        self.coefs = U_operate(U, self.coefs, axis)

In [3]:
psi = Qpsi(N = 3)
psi.build_random_vec()
coefs = psi.get_coefs()
coefs

array([0.41163601+0.67243594j, 0.5248579 +0.1001748j ,
       0.97442621+0.73694716j, 0.95292009+0.06349662j,
       0.58789914+0.20318055j, 0.54410173+0.20528513j,
       0.75719711+0.48504896j, 0.01622061+0.17764638j])

In [4]:
val = [1, 2, 3, 4]

def get_combinations(val):
    comb_list = []
    for j in range(1,len(val)+1):
        com_set = itertools.combinations(val, j)
        for i in com_set: 
            comb_list.append(i)
    return comb_list
        
print((get_combinations(val)))

[(1,), (2,), (3,), (4,), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4), (1, 2, 3, 4)]


In [5]:
i = 5
K = 7
ax = 1

bin(i&K^(1<<ax)), bin(i^(1<<ax)), bin(i&(K^(1<<ax))), bin(i), bin(K), bin(1<<ax)

('0b111', '0b111', '0b101', '0b101', '0b111', '0b10')

In [6]:

def U_operate(U, coefs, axis):
    if ((len(U) != 2**len(axis)) | (len(coefs) < len(U)) | (max(axis) > len(U))):
        print('Некорректно задано преобразование')
        return 0
    a = coefs
    b = [0]*len(a)
    
    ax = axis[0]
    
    N = len(U)  # размерность преобразования 
    K = len(coefs) # размерность состояния системы
    
    axis_comb = get_combinations(axis)
    old_ind_set = {K+1}
    
    
    for i in range(K):
        
        zero_index = (K-1)
            
        for ax in axis:
            zero_index = zero_index^(1<<ax)
            
        zero_index = i&zero_index
        
        if zero_index in old_ind_set:
            continue
            
        old_ind_set.add(zero_index)

                
        index = zero_index
        
        b[index] += a[zero_index]*U[0][0]
        
        
        u1 = 0
        for ax_list in axis_comb:
            u1 += 1
            m = 0
            for ax in ax_list:
                m += 1<<ax

            b[index] += a[zero_index^m]*U[0][u1]
            

        u0 = 0
        for ax_list in axis_comb:
            u0 += 1
            r = 0
            for ax in ax_list:
                r += 1<<ax
            
            index = zero_index^r
            old_ind_set.add(index)
            
            b[index] += a[zero_index]*U[u0][0]
            
            
            
            u1 = 0
            for ax1_list in axis_comb:
                u1 += 1
                m = 0
                for ax1 in ax1_list:
                    m += 1<<ax1
                b[index] += a[zero_index^m]*U[u0][u1]
            
        
    return b
        


In [7]:
U = [[0,1],
     [1,0]]
coefs = [1,0]
axis = [0]

start_time = time.time()
res0 = U_operate(U, coefs, axis)
print("custom function--- %s seconds ---" % (time.time() - start_time))
print(res0)

custom function--- 8.96453857421875e-05 seconds ---
[0, 1]


In [9]:
U = [[0,0,0,1],
     [0,0,1,0],
     [0,1,0,0],
     [1,0,0,0]]
coefs = [1,1,0,0]
axis = [0,1]

start_time = time.time()
res0 = U_operate(U, coefs, axis)
print("custom function--- %s seconds ---" % (time.time() - start_time))
print(res0)

custom function--- 9.679794311523438e-05 seconds ---
[0, 0, 1, 1]


In [10]:
a = 0
U = X()
U1 = X(a,3)
coefs = [1,0,2,3,0,4,0,0]
axis = [a]

print(U1.shape)

start_time = time.time()
res0 = U_operate(U, coefs, axis)
print("custom function--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
res1 = np.dot(coefs,U1)
print("np.mult--- %s seconds ---" % (time.time() - start_time))

print(res0, res1) 

(8, 8)
custom function--- 0.00016641616821289062 seconds ---
np.mult--- 0.007649898529052734 seconds ---
[0j, (1+0j), (3+0j), (2+0j), (4+0j), 0j, 0j, 0j] [0.+0.j 1.+0.j 3.+0.j 2.+0.j 4.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [11]:
axis = [1,2]
U = CX()
U1 = CX(axis,3)
coefs = [1,0,2,3,0,4,0,0]

print(U1.shape)

start_time = time.time()
res0 = U_operate(U, coefs, axis)
print("custom function--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
res1 = np.dot(coefs,U1)
print("np.mult--- %s seconds ---" % (time.time() - start_time))

res2 = np.dot(U1,coefs)

print(res0, res1, res2) 

(8, 8)
custom function--- 0.00022554397583007812 seconds ---
np.mult--- 0.00023293495178222656 seconds ---
[(1+0j), 0j, (2+0j), (3+0j), 0j, 0j, 0j, (4+0j)] [1.+0.j 0.+0.j 2.+0.j 3.+0.j 0.+0.j 0.+0.j 0.+0.j 4.+0.j] [1.+0.j 0.+0.j 2.+0.j 3.+0.j 0.+0.j 0.+0.j 0.+0.j 4.+0.j]


In [12]:
axis = [0,1,2]
U = TOFFOLI()
U1 = TOFFOLI(axis,3)
coefs = [1,0,2,3,0,4,0,0]

print(U1.shape)

start_time = time.time()
res0 = U_operate(U, coefs, axis)
print("custom function--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
res1 = np.dot(coefs,U1)
print("np.mult--- %s seconds ---" % (time.time() - start_time))

print(res0, res1) 

(8, 8)
custom function--- 0.00030541419982910156 seconds ---
np.mult--- 0.00023436546325683594 seconds ---
[(1+0j), 0j, (2+0j), (3+0j), 0j, (4+0j), 0j, 0j] [1.+0.j 0.+0.j 2.+0.j 3.+0.j 0.+0.j 4.+0.j 0.+0.j 0.+0.j]


In [25]:
N = 21

a = 0
U = X()
# U1 = X(a,N)
# coefs = [1,0,2,3,0,4,0,0]
coefs = np.random.rand(2**N)
axis = [a]

# print(U1.shape)

start_time = time.time()
res0 = U_operate(U, coefs, axis)
print("custom function--- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()
# res1 = np.dot(coefs,U1)
# print("np.mult--- %s seconds ---" % (time.time() - start_time))

# print(sum(res0 - res1) )

custom function--- 14.850182056427002 seconds ---
